In [1]:
import os
import json
import numpy as np
import pandas as pd
import polar_diagrams
from sklearn.datasets import load_wine

from dash import Dash, dcc, html, Input, Output, callback, State
from dash.exceptions import PreventUpdate
import plotly.graph_objects as go

In [2]:
df_wine_data = load_wine(return_X_y=False, as_frame=True)['data']
df_wine_data['od_diluted'] = df_wine_data['od280/od315_of_diluted_wines']
df_wine_data.drop(['od280/od315_of_diluted_wines', 'proline'], axis=1,
                  inplace=True)
df_wine_data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od_diluted
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93
...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62


In [3]:
def app_create_dashboard(df_input, string_reference_model,
                         string_diagram_type):
    dash_app = Dash("Polar Diagrams Dashboard")
    dash_app.css.config.serve_locally = True
    dash_app.scripts.config.serve_locally = True

    # TODO: Use the string_diagram_type argument
    chart_upper_left = polar_diagrams.chart_create_taylor_diagram(
        df_input, string_reference_model=string_reference_model).update_layout(
        dragmode='zoom', clickmode='event+select', hovermode=False)
    chart_upper_right = polar_diagrams.chart_create_taylor_diagram(
        df_input, string_reference_model=string_reference_model).update_layout(
        dragmode='select', clickmode='event+select')

    # We disable a legend for the second diagram
    dict_upper_right = chart_upper_right.to_dict()
    for int_i in range(len(dict_upper_right['data'])):
        dict_upper_right['data'][int_i]['showlegend'] = False
    chart_upper_right = go.Figure(dict_upper_right)

    dict_upper_left = chart_upper_left.to_dict()
    global _FLOAT_MAX_R
    _FLOAT_MAX_R = dict_upper_left['layout']['polar']['radialaxis']['range'][1]
    global _INT_MAX_THETA
    _INT_MAX_THETA = dict_upper_left['layout']['polar']['angularaxis'][
        'tickvals'][0]

    # TODO: Create a nicer layout with a title, logo, etc.
    dash_app.layout = html.Div(
        [
            html.Div(
                dcc.Graph(
                    id="upper-left",
                    figure=chart_upper_left,
                    config={
                        'modeBarButtonsToRemove': ['select', 'pan', 'lasso',
                                                   'zoomIn', 'zoomOut',
                                                   'autoScale', 'resetScale'],
                        'staticPlot': False,
                        'editSelection': True,
                        'displaylogo': False,
                        'doubleClick': False,  # TODO: Check why this doesn't work
                        'showAxisDragHandles': False}),
                className="two-columns",
                style={'width': '50%','display': 'inline-block'},
            ),
            html.Div(
                dcc.Graph(
                    id="upper-right",
                    figure=chart_upper_right,
                    config={
                        'modeBarButtonsToRemove': ['zoom', 'pan', 'lasso',
                                                   'zoomIn', 'zoomOut',
                                                   'select', 'autoScale',
                                                   'resetScale'],
                        'displaylogo': False,
                        'doubleClick': False,
                        'showAxisDragHandles': False}),
                className="two-columns",
                style={'width': '50%','display': 'inline-block'},
            ),
        ],
        className="row",
    )

    dash_app.run(debug=True, jupyter_mode='external')

    return None


@callback(
    Output(component_id="upper-left", component_property="figure", allow_duplicate=True),
    Output(component_id="upper-right", component_property="figure", allow_duplicate=True),
    Input(component_id="upper-left", component_property="restyleData"),
    State('upper-left', 'figure'),
    State('upper-right', 'figure'),
    prevent_initial_call=True,
)
def list_update_legends(list_legend_points, dict_upper_left, dict_upper_right):
    if list_legend_points:
        #print(json.dumps(list_legend_points, indent=2))
        '''
        [
          {
            "visible": [
              "legendonly"
            ]
          },
          [
            10
          ]
        ]
        [
          {
            "visible": [
              true
            ]
          },
          [
            1
          ]
        ]
        '''
        for int_i, int_legend_point in enumerate(list_legend_points[1]):
            for int_j, dict_one_trace in enumerate(dict_upper_right['data']):
                if dict_one_trace['name'].startswith(str(int_legend_point)):
                    if isinstance(
                        list_legend_points[0]['visible'][int_i], bool) and (
                        list_legend_points[0]['visible'][int_i], bool == True):
                        dict_upper_right['data'][int_j]['visible'] = True
                    else:
                        dict_upper_right['data'][int_j]['visible'] = False

        chart_upper_left = go.Figure(dict_upper_left)
        chart_upper_right = go.Figure(dict_upper_right)

        return chart_upper_left, chart_upper_right


@callback(
    Output(component_id="upper-left", component_property="figure"),
    Output(component_id="upper-right", component_property="figure"),
    Input(component_id="upper-left", component_property="relayoutData"),
    State('upper-left', 'figure'),
    State('upper-right', 'figure'),
    prevent_initial_call=True
)
def list_update_zooms(dict_box_selected_points, dict_upper_left,
                      dict_upper_right):

    if dict_box_selected_points and (
            'polar.radialaxis.range' in dict_box_selected_points):

        #print(json.dumps(dict_box_selected_points, indent=2))
        dict_radial_range = dict_box_selected_points['polar.radialaxis.range']

        try:
            # TODO: Catch doubleclick event and do this as well
            for int_i, trace in enumerate(dict_upper_left['data']):
                if 'name' in trace and trace['name'] == 'Selection':
                    del dict_upper_left['data'][int_i]

            # We create a circular rectangle of 60 points by creating them and
            # connecting them with a line
            np_alpha = np.linspace(0, _INT_MAX_THETA, 60).tolist()
            np_selection_theta = np_alpha + np_alpha[::-1] + [np_alpha[0]]

            chart_upper_left = go.Figure(dict_upper_left)
            chart_upper_right = go.Figure(dict_upper_right)

            chart_upper_left.add_trace(
                go.Scatterpolar(r=[dict_radial_range[0]]*60 +\
                                  [dict_radial_range[1]]*60 +\
                                  [dict_radial_range[0]],
                                theta =np_selection_theta,
                                name='Selection',
                                fill='toself',
                                mode='lines',
                                showlegend=False,
                                line=dict(
                                    color='lightgrey',
                                    dash='dot',
                                    width=3,
                                )))

            chart_upper_left['layout']['polar']["radialaxis"][
                "autorange"] = False
            chart_upper_left['layout']['polar']["radialaxis"][
                'rangemode'] = 'normal'
            chart_upper_right['layout']['polar']["radialaxis"][
                "autorange"] = False
            chart_upper_right['layout']['polar']["radialaxis"][
                'rangemode'] = 'normal'

            chart_upper_left['layout']['polar']["radialaxis"]["range"] = [
                0, _FLOAT_MAX_R]
            chart_upper_right['layout']['polar']["radialaxis"]["range"] = [
                dict_radial_range[0], dict_radial_range[1]]
            pass
        except (KeyError, TypeError):
            pass

    else:
        raise PreventUpdate

    return chart_upper_left, chart_upper_right


In [4]:
app_create_dashboard(df_wine_data, 'alcohol', 'taylor')

Dash app running on http://127.0.0.1:8050/
